In [54]:
import os, sys, keras
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Conv2D, AveragePooling2D, Dropout, Activation, Input, concatenate, Flatten, Dense
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, CSVLogger

In [55]:
(trainX, trainY), (testX, testY) = cifar10.load_data()

In [56]:
IS_POSITION_BASED = True
# IS_POSITION_BASED = int(sys.argv[1])
image_size = 32
batch_size = 10
image_channels = 3
if IS_POSITION_BASED:
    image_channels = 5
num_classes = 10
epochs = 1

In [57]:

trainX.shape

(50000, 32, 32, 3)

In [58]:
if IS_POSITION_BASED:
    nx, ny = (32, 32)
    xt = np.linspace(0, 1, nx)
    yt = np.linspace(0, 1, ny)
    xpos, ypos = np.meshgrid(xt, yt)
    trainX = np.swapaxes(np.swapaxes(trainX, 0, 3), 1, 3)
    trainX = np.swapaxes(np.swapaxes(np.swapaxes(np.vstack([[trainX[0]], [trainX[1]], [trainX[2]], [[xpos,]*trainX.shape[1]], [[ypos,]*trainX.shape[1]]]), 0, 2), 0, 1), 2, 3)
    testX = np.swapaxes(np.swapaxes(testX, 0, 3), 1, 3)
    testX = np.swapaxes(np.swapaxes(np.swapaxes(np.vstack([[testX[0]], [testX[1]], [testX[2]], [[xpos,]*testX.shape[1]], [[ypos,]*testX.shape[1]]]), 0, 2), 0, 1), 2, 3)

In [59]:
trainY = keras.utils.to_categorical(trainY, num_classes)
testY = keras.utils.to_categorical(testY, num_classes)

In [60]:
# normalize data
trainX = trainX.astype('float32')/255.0
testX = testX.astype('float32')/255.0

In [61]:
main_input = Input(shape=(image_size, image_size, image_channels), name="main_input")

layer1 = Conv2D(16, (3,3), padding='same')(main_input)
layer1 = BatchNormalization()(layer1)
layer1 = Activation('relu')(layer1)

layer2b = Conv2D(32, (1,1), padding='same')(layer1)
layer2a = Conv2D(32, (3,3), padding='same')(layer1)
layer2a = BatchNormalization()(layer2a)
layer2a = Activation('relu')(layer2a)
layer2a = Conv2D(32, (3,3), padding='same')(layer2a)
layer2 = concatenate([layer2a, layer2b])


layer3 = BatchNormalization()(layer2)
layer3 = Activation('relu')(layer3)
layer3 = Conv2D(32, (3,3), padding='same')(layer3)
layer3 = BatchNormalization()(layer3)
layer3 = Activation('relu')(layer3)
layer3 = Conv2D(32, (3, 3), padding='same')(layer3)
layer3 = concatenate([layer3, layer2])

layer3 = BatchNormalization()(layer3)
layer3 = Activation('relu')(layer3)


layer4b = Conv2D(64, (1,1), padding='same')(layer3)
layer4a = Conv2D(64, (3, 3), padding='same')(layer3)
layer4a = BatchNormalization()(layer4a)
layer4a = Activation('relu')(layer4a)
layer4a = Conv2D(64, (3, 3), padding='same')(layer4a)
layer4 = concatenate([layer4a, layer4b])

layer4 = BatchNormalization()(layer4)
layer4 = Activation('relu')(layer4)
layer4 = AveragePooling2D((8,8), strides=1)(layer4)


out = Flatten(name='flatten')(layer4)
out = Dense(4096, activation='relu', name='fc1')(out)
out = Dropout(0.25)(out)
out = Dense(1024, activation='relu', name='fc2')(out)
out = Dropout(0.25)(out)
out = Dense(10, activation='relu', name='output')(out)

model = Model(input=main_input, output=out)
# model.summary()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:46: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ou..., inputs=Tensor("ma...)`


In [62]:
# Let's train the model using Adam
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [63]:
if not os.path.exists("../results"):
    os.mkdir("../results")
    os.mkdir("../results/position")
    os.mkdir("../results/position/best_models")
    os.mkdir("../results/normal")
    os.mkdir("../results/normal/best_models")

In [ ]:
# checkpoints and logs

if IS_POSITION_BASED:
    checkpointer = ModelCheckpoint(filepath="../results/position/best_models/fn_model.{epoch:02d}-{val_acc:.6f}.hdf5", verbose=1, monitor='val_acc', save_best_only=True, save_weights_only=False, mode='max', period=1)
    tf_board = TensorBoard(log_dir='../results/position/logs', histogram_freq=0, write_graph=True, write_images=True)
    csv_logger = CSVLogger('../results/position/training.log')
else :
    checkpointer = ModelCheckpoint(filepath="../results/normal/best_models/fn_model.{epoch:02d}-{val_acc:.6f}.hdf5", verbose=1, monitor='val_acc', save_best_only=True, save_weights_only=False, mode='max', period=1)
    tf_board = TensorBoard(log_dir='../results/normal/logs', histogram_freq=0, write_graph=True, write_images=True)
    csv_logger = CSVLogger('../results/normal/training.log')

early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
# training network
model.fit(trainX, trainY, batch_size=batch_size, epochs=epochs, validation_split=0.05,
              callbacks=[early_stopping, checkpointer, tf_board, csv_logger], shuffle=True)

Train on 47500 samples, validate on 2500 samples


In [ ]:
# testing network
if IS_POSITION_BASED:
    model.save("./results/position/best_models/final_model.hdf5")
    score, acc = model.evaluate(testX, testY, batch_size=batch_size)
    resultsfile = open("./results/position/results.txt", 'w')
    resultsfile.write("test_acc: "+str(acc)+"\n")
    resultsfile.write("test_score: " + str(score))
    resultsfile.close()
else:
    model.save("./results/normal/best_models/final_model.hdf5")
    score, acc = model.evaluate(testX, testY, batch_size=batch_size)
    resultsfile = open("./results/normal/results.txt", 'w')
    resultsfile.write("test_acc: "+str(acc)+ "\n")
    resultsfile.write("test_score: "+str(score))
    resultsfile.close()